In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [9]:
listings['amenities_set'] = listings['amenities'].fillna('{}').map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities'].fillna('{}').map(string_to_set).index:
    all_amenities = all_amenities.union(listings['amenities'].fillna('{}').map(string_to_set)[idx])

In [10]:
all_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [5]:
listings['amenities_set'] = listings['amenities'].fillna('{}').map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities'].fillna('{}').map(string_to_set).index:
    all_amenities = all_amenities.union(listings['amenities'].fillna('{}').map(string_to_set)[idx])
    
for amen in all_amenities:
        
    if len(amen.split(' ')) == 1:
        listings['has_' + amen] = 0
        listings['has_' + amen] = listings['amenities_set'].map(lambda x: has_amenity(x, amen))
        continue
            
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    listings['has_' + amen] = 0
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: has_amenity(x, amen))
        
    
has_amenties_list = []
for amen in all_amenities:
        
    if len(amen.split(' ')) == 1:
        has_amenties_list.append('has_' + amen)
        continue
            
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    has_amenties_list.append('has_' + amen)
    

In [6]:
listings[has_amenties_list].isnull().sum()

has_                                              6525
has_Other_pet(s)                                  6608
has_Essentials                                    1061
has_Carbon_Monoxide_Detector                      6608
has_Elevator_in_Building                          6608
has_Indoor_Fireplace                              6608
has_translation_missing:_en.hosting_amenity_50    6608
has_Internet                                      1852
has_Washer                                        1416
has_Hangers                                       3194
has_Buzzer/Wireless_Intercom                      6608
has_TV                                            1334
has_Gym                                           5634
has_Fire_Extinguisher                             6608
has_Hot_Tub                                       6608
has_Cable_TV                                      6608
has_Air_Conditioning                              6608
has_Laptop_Friendly_Workspace                     6608
has_Suitab

In [ ]:
listings[has_amenties_list] = listings[has_amenties_list].fillna(0)

In [7]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])
        
for w in all_host_verifications:
        
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = 0
    listings['uses_' + w] = a.map(lambda x: has_amenity(x, w))
        
    
uses_verification_list = []
for veri in all_host_verifications:
        
    if '' in veri:
        veri = veri.strip()[1:-1].replace(' ', '_')
            
    uses_verification_list.append('uses_' + veri)
        


In [8]:
listings[uses_verification_list].isnull().sum()

uses_phone             6608
uses_linkedin          6608
uses_email             6608
uses_manual_online     6608
uses_facebook          6608
uses_amex              6608
uses_phone             6608
uses_sent_id           6608
uses_jumio             6608
uses_google            6608
uses_weibo             6608
uses_manual_offline    6608
uses_kba               6608
uses_kba               6608
uses_reviews           6608
dtype: int64

In [ ]:
listings[uses_verification_list] = listings[uses_verification_list].fillna(0)

In [ ]:
listings[has_amenties_list].isnull().sum()

In [7]:
listings[uses_verification_list].head(4)

,uses_phone,uses_linkedin,uses_email,uses_manual_online,uses_facebook,uses_amex,uses_phone,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_kba,uses_reviews
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
listings[has_amenties_list].head(4)

,has_,has_Other_pet(s),has_Essentials,has_Carbon_Monoxide_Detector,has_Elevator_in_Building,has_Indoor_Fireplace,has_translation_missing:_en.hosting_amenity_50,has_Internet,has_Washer,has_Hangers,...,has_Pets_Allowed,has_Wheelchair_Accessible,has_First_Aid_Kit,has_Dog(s),has_Wireless_Internet,has_Pool,has_Breakfast,has_Smoking_Allowed,has_Cat(s),has_24-Hour_Check-in
0,0.0,0,1.0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0.0,0,0.0,0
1,0.0,0,1.0,0,0,0,0,1.0,0.0,0.0,...,0,0,0,0.0,0,1.0,0.0,0,0.0,0
2,0.0,0,1.0,0,0,0,0,0.0,1.0,0.0,...,0,0,0,0.0,0,1.0,0.0,0,0.0,0
3,0.0,0,1.0,0,0,0,0,1.0,1.0,1.0,...,0,0,0,0.0,0,0.0,0.0,0,0.0,0
